# Gaussian Transformation

This tutorial demonstrates the basic working principles of PennyLane for Continuous Variable (CV) photonic devices. For more details about photonic quantum computing, the [Strawberry Fields](https://strawberryfields.readthedocs.io/en/latest/) documentation is a great starting point.

<font color=brown> **GOAL:** To optimize displacement gate parameters in order to displace a *vacuum mode* in the phase space so as to get one *qumode*.</font>

### The Quantum Circuit

For this basic example, we will consider a special subset of CV operations: *Gaussian transformations*. We work with the following simple Gaussian circuit:

What is this circuit doing?

1. **We begin with one wire (qumode) in the vacuum state.** Note that we use the same notation $\mid0\rangle$ for the initial state as in the qubit rotation example. In a photonic CV system, this state is the vacuum state i.e., the average photon number in the wire is zero.
2. **We displace the qumode.** The displacement gate linearly shifts the state of the qumode in phase space. The vacuum state is centered at the origin in phase space while the displaced state will be centered at the point $\alpha$.
3. **We rotate the qumode.** This is another linear transformation in phase space, albeit a rotation (by angle $\phi$) instead of a displacement.
4. **Finally, we measure the mean photon number $\langle\hat{n}\rangle=\langle\hat{a}^{\dagger}\hat{a}\rangle$.** This quantity, which tells us the average amount of photons in the final state, is proportional to the energy of the photonic system.

We want to optimize the circuit parameters $(\alpha,\phi)$ such that the mean photon number is equal to one. The rotation gate is actually a *passive transformation* meaning that it does not change the energy of the system. The displacement gate is an *active transformation* which modifies the energy of the photonic system.

### PennyLane Execution

In [1]:
# lets first import the essentials
import pennylane as qml
from pennylane import numpy as np

In [2]:
#create a device
dev_gaussian = qml.device('default.gaussian', wires=1)

In [3]:
#construct a QNode
@qml.qnode(dev_gaussian)
def mean_photon_gaussian(mag_alpha, phase_alpha, phi):
    qml.Displacement(mag_alpha, phase_alpha, wires=0)
    qml.Rotation(phi, wires=0)
    return qml.expval.MeanPhoton(0)

As we want the mean photon number to be exactly one, we will use a squared-difference cost function:

In [4]:
def cost(params):
    return (mean_photon_gaussian(params[0], params[1], params[2]) - 1.) ** 2

At the beginning of the optimization, we choose arbitrary small initial parameters:

In [5]:
init_params = [0.015, 0.02, 0.005]
cost(init_params)

0.9995500506249999

When the gate parameters are near to zero, the gates are close to the *identity transformation*, which leaves the initial state largely unchanged. Since the initial state contains no photons, the mean photon number of the circuit output is approximately zero and the cost is close to one.

<font color=brown> **NOTE:** We avoided initial parameters which are exactly zero because that corresponds to a critical point with zero gradient.</font>

Now, let’s use the `GradientDescentOptimizer` and update the circuit parameters over 20 optimization steps:

In [6]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.1)

In [7]:
# set the number of steps
steps = 20
# set the initial parameter values
params = init_params

In [8]:
for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)
    #print cost after every step
    print('Cost after step {:5d}: {:8f}'.format(i+1, cost(params)))

Cost after step     1: 0.999118
Cost after step     2: 0.998273
Cost after step     3: 0.996618
Cost after step     4: 0.993382
Cost after step     5: 0.987074
Cost after step     6: 0.974837
Cost after step     7: 0.951332
Cost after step     8: 0.907043
Cost after step     9: 0.826649
Cost after step    10: 0.690812
Cost after step    11: 0.490303
Cost after step    12: 0.258845
Cost after step    13: 0.083224
Cost after step    14: 0.013179
Cost after step    15: 0.001001
Cost after step    16: 0.000049
Cost after step    17: 0.000002
Cost after step    18: 0.000000
Cost after step    19: 0.000000
Cost after step    20: 0.000000


Try this yourself — the optimization should converge after about 20 steps to a cost function value of zero; corresponding to the following final values for the parameters:

In [9]:
print('Optimized mag_alpha:{:8f}'.format(params[0]))
print('Optimized phase_alpha:{:8f}'.format(params[1]))
print('Optimized phi:{:8f}'.format(params[2]))

Optimized mag_alpha:0.999994
Optimized phase_alpha:0.020000
Optimized phi:0.005000


We observe that the two angular parameters `phase_alpha` and `phi` do not change during the optimization. This makes sense as only the magnitude of the complex displacement `mag_alpha` affects the mean photon number of the circuit.